In [1]:
import pandas as pd
import numpy as np
import h5py

# Data processing

In [2]:
cleaned_path = './1 - Organized data gauge/BRAZIL/DATASETS/BRAZIL_DAILY_1961_2024_CLEANED.h5'
with h5py.File(cleaned_path, 'r') as hdf:
    keys = list(hdf.keys())
    print(keys)

['table_data', 'table_data_filtered', 'table_info', 'table_p_availability', 'table_preclassif', 'table_q1_gaps', 'table_q2_week', 'table_q3_outliers', 'table_qc_info']


In [ ]:
df_data = pd.read_hdf(cleaned_path, key = 'table_data_filtered', encoding = 'utf-8')
df_data.head()

In [ ]:
df_info = pd.read_hdf(cleaned_path, key = 'table_info', encoding = 'utf-8')
df_info.head()

,name_station,gauge_code,city,state,responsible,source,state_abbreviation,lat,long
18981,SELVIRIA | S717,S717,SELVIRIA,MATO GROSSO DO SUL,INMET,INMET,MS,-20.351389,-51.430278
18980,SANTA RITA DO PARDO | S716,S716,SANTA RITA DO PARDO,MATO GROSSO DO SUL,INMET,INMET,MS,-21.305889,-52.820375
18979,RIBAS DO RIO PARDO | S715,S715,RIBAS DO RIO PARDO,MATO GROSSO DO SUL,INMET,INMET,MS,-20.466694,-53.763028
18978,PEDRO GOMES | S714,S714,PEDRO GOMES,MATO GROSSO DO SUL,INMET,INMET,MS,-18.072778,-54.548889
18977,NOVA ANDRADINA | S713,S713,NOVA ANDRADINA,MATO GROSSO DO SUL,INMET,INMET,MS,-22.078611,-53.465833


In [ ]:
df_complete_info = pd.merge(df_data, df_info, on = 'gauge_code', how = 'inner')
del df_data, df_info
df_complete_info['year'] = df_complete_info['datetime'].dt.year
df_complete_info

,gauge_code,datetime,rain_mm,name_station,city,state,responsible,source,state_abbreviation,lat,long,year
0,00047000,1961-01-01,0.0,SALINÓPOLIS,SALINÓPOLIS,PARÁ,INMET,HIDROWEB,PA,-0.650000,-47.550000,1961
1,00047000,1961-01-02,0.0,SALINÓPOLIS,SALINÓPOLIS,PARÁ,INMET,HIDROWEB,PA,-0.650000,-47.550000,1961
2,00047000,1961-01-03,0.0,SALINÓPOLIS,SALINÓPOLIS,PARÁ,INMET,HIDROWEB,PA,-0.650000,-47.550000,1961
3,00047000,1961-01-04,0.0,SALINÓPOLIS,SALINÓPOLIS,PARÁ,INMET,HIDROWEB,PA,-0.650000,-47.550000,1961
4,00047000,1961-01-05,0.0,SALINÓPOLIS,SALINÓPOLIS,PARÁ,INMET,HIDROWEB,PA,-0.650000,-47.550000,1961
...,...,...,...,...,...,...,...,...,...,...,...,...
123593340,S717,2021-12-27,0.0,SELVIRIA | S717,SELVIRIA,MATO GROSSO DO SUL,INMET,INMET,MS,-20.351389,-51.430278,2021
123593341,S717,2021-12-28,0.0,SELVIRIA | S717,SELVIRIA,MATO GROSSO DO SUL,INMET,INMET,MS,-20.351389,-51.430278,2021
123593342,S717,2021-12-29,0.0,SELVIRIA | S717,SELVIRIA,MATO GROSSO DO SUL,INMET,INMET,MS,-20.351389,-51.430278,2021
123593343,S717,2021-12-30,0.0,SELVIRIA | S717,SELVIRIA,MATO GROSSO DO SUL,INMET,INMET,MS,-20.351389,-51.430278,2021


In [ ]:
max(df_complete_info['rain_mm'])

In [ ]:
def calculateP(df):
    # Group by gauge_code and year, then count the number of rows for each group
    df_p = df.groupby(['gauge_code', 'year']).size().reset_index(name='active_days')
    return df_p

# Example usage
df_p_availability = calculateP(df_complete_info)

years = pd.Series(range(1961, 2025), name='year')
# Calculate the number of days in each year
day_count = years.apply(lambda x: 366 if (x % 4 == 0 and (x % 100 != 0 or x % 400 == 0)) else 365)
# Create the DataFrame
df_years = pd.DataFrame({'year': years, 'day_count': day_count})
del years, day_count


df_p_availability = pd.merge(df_years, df_p_availability, on = 'year', how = 'left')
del df_years

df_p_availability['p_availability'] = df_p_availability['active_days'] / df_p_availability['day_count'] * 100
df_p_availability = df_p_availability[['gauge_code', 'year', 'p_availability']]
df_p_availability.to_hdf(cleaned_path, key = 'table_p_availability', encoding = 'utf-8', mode='r+')
df_p_availability

,gauge_code,year,p_availability
0,00047000,1961,100.000000
1,00048000,1961,100.000000
2,00062000,1961,100.000000
3,00065000,1961,8.493151
4,00067000,1961,100.000000
...,...,...,...
345863,B803,2024,100.000000
345864,B804,2024,100.000000
345865,B806,2024,100.000000
345866,B807,2024,100.000000


In [11]:
with h5py.File(cleaned_path, 'r') as hdf:
    keys = list(hdf.keys())
    print(keys)

['table_data', 'table_data_filtered', 'table_info', 'table_p_availability', 'table_preclassif']


In [10]:
df_teste = pd.read_hdf(cleaned_path, key = 'table_p_availability', encoding = 'utf-8')
df_teste

,gauge_code,year,p_availability
0,00047000,1961,100.000000
1,00048000,1961,100.000000
2,00062000,1961,100.000000
3,00065000,1961,8.493151
4,00067000,1961,100.000000
...,...,...,...
345863,B803,2024,100.000000
345864,B804,2024,100.000000
345865,B806,2024,100.000000
345866,B807,2024,100.000000
